In [1]:
import Pkg; 

cd(joinpath(@__DIR__, "../../"))
Pkg.activate("Project.toml")

using MorphoMol
using PyCall
using JLD2
using LinearAlgebra
using Rotations
using GLMakie

  Activating project at `~/Doktor/Code/MorphoMol/MorphoMolHelpers`
Precompiling GLMakie
┌ Warning: attempting to remove probably stale pidfile
│   path = "/Users/ivanspirandelli/.julia/compiled/v1.10/Makie/iZ1Bl_cwiIy.ji.pidfile"
└ @ FileWatching.Pidfile ~/.julia/juliaup/julia-1.10.4+0.aarch64.apple.darwin14/share/julia/stdlib/v1.10/FileWatching/src/pidfile.jl:244
  ✓ Makie
  ✓ GLMakie
  2 dependencies successfully precompiled in 58 seconds. 239 already precompiled.
[ Info: Precompiling GLMakie [e9467ef8-e4e7-5192-8a1a-b1aee30e663a]
Precompiling DistributionsTestExt
  ✓ Distributions → DistributionsTestExt
  1 dependency successfully precompiled in 7 seconds. 43 already precompiled.
[ Info: Precompiling DistributionsTestExt [ffbe0ea5-a612-5ff7-aaf5-cac02eef3019]
┌ Warning: Module Distributions with build ID fafbfcfd-c03e-36b5-0000-fd6c49c103b3 is missing from the cache.
│ This may mean Distributions [31c24e10-a181-5473-b8eb-7969acd0382f] does not support precompilation but is imported 

In [2]:
py"""
import oineus as oin
import numpy as np
import torch
import diode

def get_diagram(points, dim):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])
    # no cohomology
    dualize = False
    # create VRU decomposition object, does not perform reduction yet
    dcmp = oin.Decomposition(fil, dualize)
    rp = oin.ReductionParams()
    rp.compute_u = rp.compute_v = True
    rp.n_threads = 1
    # perform reduction
    dcmp.reduce(rp)
    dgm_dim = dcmp.diagram(fil, include_inf_points=False)[dim]
    return dgm_dim
"""

function get_diagram(points::Vector{Vector{Float64}}, dim::Int = 1)
    py"get_diagram"(points, dim)
end

function get_total_persistence(dgm::Matrix{Float64})
    sum(dgm[:,2] - dgm[:,1])
end


get_total_persistence (generic function with 1 method)

In [3]:
function txt_to_points(input_file)
    [[parse(Float64, s) for s in split(p," ")] for p in readlines(input_file)[3:end]]
end

txt_to_points (generic function with 1 method)

In [4]:
function get_diagrams(folder)
    files = readdir(folder)
    n = length(files)
    [get_diagram(txt_to_points("$(folder)$(i-1).txt")) for i in 1:n]
end

function get_tps(dgms)
    [get_total_persistence(dgm) for dgm in dgms]
end

function get_fsols(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[6]) for i in 1:n]
end

function get_thetas(folder)
    files = readdir(folder)
    n = length(files)
    [parse(Float64, split(readlines("$(folder)$(i-1).txt")[2], " ")[7]) for i in 1:n]
end

get_thetas (generic function with 1 method)

In [5]:
function folder_of_txts_to_jld2_with_persistence_computation(folder)
    mol_type = split(folder, "/")[end-2]
    idx = split(folder, "/")[end-1]
    dgms = get_diagrams(folder)
    tps = get_tps(dgms)
    fsols = get_fsols(folder)
    thetas = get_thetas(folder)
    @save "../../Data/measure_paths/jld2s/$(mol_type)/$(idx).jld2" dgms tps fsols thetas
end

folder_of_txts_to_jld2_with_persistence_computation (generic function with 1 method)

In [7]:
# folder = "../../Data/measure_paths/txts/6r7m/"
# for subdir in readdir(folder)
#     if subdir != ".DS_Store"
#         folder_of_txts_to_jld2_with_persistence_computation("$(folder)$(subdir)/")
#     end
# end

In [102]:
# folder = "../../Data/measure_paths/txts/6r7m/1/"
# dgms = get_diagrams(folder)
# xs = [i for i in 1:length(dgms)]
# tps = get_tps(dgms)
# fsols = get_fsols(folder)
# thetas = get_thetas(folder);

@load "../../Data/measure_paths/jld2s/6r7m/11.jld2" dgms tps fsols thetas
xs = [i for i in 1:length(dgms)];

In [105]:
f = Figure(fontsize = 7)
pd_ax = Axis(f[1, 1:3], title = "Persistence Diagram")
tp_ax = Axis(f[2, 1], title = "Total Persistence")
theta_ax = Axis(f[2, 2], title = L"\Theta")
fsol_ax = Axis(f[2, 3], title = L"F_{sol}")

sl_i = Slider(f[3, 1:3], range = 1:length(dgms), startvalue = 1)

x = sl_i.value

tp_mark = @lift(Point2f($x, $@lift(tps[$x])))
theta_mark = @lift(Point2f($x, $@lift(thetas[$x])))
fsol_mark = @lift(Point2f($x, $@lift(fsols[$x])))

dgm_points = @lift([Point2f(dgms[$x][i,1], dgms[$x][i,2]) for i in 1:size(dgms[$x])[1]])

ms = 5
scatter!(pd_ax, dgm_points, markersize = ms)

scatter!(tp_ax, xs, tps, color=:red, markersize = ms)
scatter!(tp_ax, tp_mark, color = :black, markersize = 10)

scatter!(theta_ax, xs, thetas, color=:magenta, markersize = ms)
scatter!(theta_ax, theta_mark, color = :black, markersize = 10)

scatter!(fsol_ax, xs, fsols, color=:orange, markersize = ms)
scatter!(fsol_ax, fsol_mark, color = :black, markersize = 10)

display(f)


GLMakie.Screen(...)

: 

In [34]:
py"""
import oineus as oin
import numpy as np
import torch
import diode

def get_coboundary_diagram(points, n_atoms_per_mol):
    points = np.asarray(points)
    simplices = diode.fill_alpha_shapes(points)
    fil = oin.Filtration_double([oin.Simplex_double(s[0], s[1]) for s in simplices])

    def is_multi(sigma):
        has_a = has_b = False
        for v in sigma.vertices:
            if v < n_atoms_per_mol:
                has_a = True
            else:
                has_b = True
        return has_a and has_b


    cb_m = fil.coboundary_matrix()
    m = [cb_m[i] for i, sigma in enumerate(fil.cells()) if is_multi(sigma) ]
    # second argument: number of rows
    dcmp = oin.Decomposition(m, fil.size())
    params = oin.ReductionParams()
    params.clearing_opt = False
    dcmp.reduce(params)
    dgm = dcmp.diagram(fil, include_inf_points=False)
    return dgm[1], dgm[2]
"""

function get_coboundary_diagram(points, n_atoms_per_mol)
    py"get_coboundary_diagram"(points, n_atoms_per_mol)
end

get_coboundary_diagram (generic function with 2 methods)

In [32]:
@load "../../Data/jld2/single_subunits/6r7m_protor.jld2" template_mol template_radii x_init
n = length(template_radii)

1206

In [40]:
txt = "../../Data/measure_paths/txts/6r7m/1/0.txt"
dgm1, dgm2 = get_coboundary_diagram(txt_to_points(txt), n)
dgm1

268×2 Matrix{Float64}:
     0.373773  0.0
     0.374311  0.0
     0.37465   0.0
     0.375309  0.0
     0.377275  0.0
     0.377684  0.0
     0.377684  0.0
     0.378112  0.0
     0.376807  0.0
     0.378722  0.0
     0.378722  0.0
     0.378873  0.0
     0.378892  0.0
     ⋮         
   154.638     0.0
   168.614     0.0
   179.706     0.0
   182.744     0.0
   198.323     0.0
   240.791     0.0
  2836.32      0.0
  3213.39      0.0
  4020.97      0.0
  4050.56      0.0
 54533.0       0.0
 59696.2       0.0